## Tutorial 2) Readout sequences

## 0. Introduction

In the first tutorial sequence writing and parameterization with configuration files was demonstrated.
This enables user to apply and quantify any waveform on the instrument outputs.

Playing arbitrary waveforms is one part of qubit experimetns, reading them out is equally crucial.
In this tutorial we explain the concept of `ReadSequence`s and how to use them. 

The tutorial is structured in the following way:
1. The basic `ReadSequence` architecture
2. Configuring a dummy readout sequence
3. Compiling the Sequence to QUA code

## 1) The basic `ReadSequence` architecture

`ReadSequence`\
The ReadSeqeunce is a child class of the SubSequence which we exlpored in Tutorial 1. Arbitrary waveforms can still be played, parameterized and swept.
The ReadSequence is meant to handle measurement sequences and describe them in a device agnostic way. For this the classes in the rest of this list are introduced. They all relate to a single given ReadSequence.

`Signal`\
Signals represent one or more measurement results from a single physical entity (e.g qubit, SET, quantum dot etc.) A ReadSequence can have an arbitrary number of Signal instances.

`Observable`\
Each observable handles a single result that is being accquired during the execution of a ReadSequence. Observables store the qua variables that store measurement results temporarily and the streams they are saved to. Each result is assigned to a single signal instance.

`ReadoutPoint`\
A readout point describes a direct readout of a given quantum element at a differnet 'point' (e.g in voltage or frequency space).
Qua commands describing this measurement are given in the `qua_measure` method.
Per Readout point, multiple observables can be instruduced whose FPGA variables are automatically declared, assigned and saved to the correct stream by this class.

`AbstractReadout`\
The AbstractReadout works similarly to the ReadoutPoint but gives you all the freedom. Qua commands can be executed, arbitrary arguments like observables from other AbstractReadouts or even ReadoutPoints can be passed as arguments to process them further. in comparison to the ReadoutPoint, the results (observables) of an AbstractReadout can be assigned to ANY given signal.

This might seem a bit abstract on the first glace but will be a lot more clear after looking how this works in action.

<img src="images/readout_sequence_scheme.svg">

This scheme can be further visualized by comparing this with the configuration we will use in this tutorial.

## 2) Configuring a dummy readout sequence

In [5]:
readout_sequence_config = {
    'parameters': {
        't_between_measurements': {
            'value': 50,
            'unit': 'cycles',
        }
    },
    'signals':{
        'qubit1':{
            'elements': {
                'sensor1': 'readout_element',
            },
            'readout_points': {
                'ref': {
                    'method': 'average',
                    'desc':'reference point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                },
                'read': {
                    'method': 'average',
                    'desc': 'redout point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                }
            }
        },
    },
    'readout_groups': {
        'difference': {
            'qubit1__diff': {
                'method': 'difference',
                'name': 'diff',
                'args': {
                    'signal': 'qubit1',
                    'minuend': 'qubit1.ref.sensor1_IQ',
                    'subtrahend': 'qubit1.read.sensor1_IQ',
                },
            },
        }
    },
}

In [1]:
from arbok_driver import ArbokDriver, Sample, Sequence

2024-04-09 17:36:33,837 - qm - INFO     - Starting session: 47eea29e-e49e-4be3-a15a-0f1c06237b94


In [2]:
from example_configs.dummy_sample import dummy_sample
from example_sequences.dummy_readout import DummyReadout

In [3]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)
# dummy_sequence = Sequence('dummy_squence', dummy_sample)
# qm_driver.add_sequence(dummy_sequence)

In [4]:
DummyReadout??

Init signature: DummyReadout(*args: 'Any', **kwargs: 'Any') -> 'Any'
Source:        
class DummyReadout(ReadSequence):
    """
    Class containing parameters and sequence for a dummy read sequnce
    """
    def __init__(
            self,
            name: str,
            sample: Sample,
            sequence_config: dict,
        ):
        """
        Constructor method for 'DummyReadout' class
        
        Args:
            name (str): name of the sequence
            sample (Sample): Sample being used
            sequence_config (dict): Dict configuring sequence
        """
        super().__init__(
            name, sample, sequence_config,
            {'difference': Difference},
            {'average': Average}
            )

    def qua_sequence(self):
        """QUA sequence to perform mixed down up initialization"""
        qua.align()

        ref_points = [x for x in self.readout_points.values() if x.point_name == 'ref']
        for readout_point in ref_points:
       

In [6]:
dummy_readout = DummyReadout('dummy_readout', dummy_sample, readout_sequence_config)
dummy_sequence.add_subsequence(dummy_readout)

NameError: name 'dummy_sequence' is not defined

## 3) Compiling a `ReadoutSequence` to QUA code

In [ ]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    'readout_qua_program.py', qua_program)
import compiled_readout_qua_program
compiled_readout_qua_program??